# Generates images from text prompts.

By Katherine Crowson (https://github.com/crowsonkb, https://twitter.com/RiversHaveWings). It uses a 602M parameter diffusion model trained on Conceptual 12M. See the GitHub repo for more information: https://github.com/crowsonkb/v-diffusion-pytorch.

In [ ]:
# @title Licensed under the MIT License

# Copyright (c) 2022 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [ ]:
# Check the GPU

!nvidia-smi

### Install dependencies (no need to rerun this section if you restart the notebook runtime)

In [ ]:
# Install dependencies

!git clone --recursive https://github.com/crowsonkb/v-diffusion-pytorch
%pip install git+https://github.com/crowsonkb/k-diffusion

If the "Restart runtime" button appears above, please click it to restart the runtime, then continue below.

In [ ]:
# Download the diffusion model
# SHA-256: 4fc95ee1b3205a3f7422a07746383776e1dbc367eaf06a5b658ad351e77b7bda

!mkdir v-diffusion-pytorch/checkpoints
!curl -L --http1.1 https://the-eye.eu/public/AI/models/v-diffusion/cc12m_1_cfg.pth > v-diffusion-pytorch/checkpoints/cc12m_1_cfg.pth

### Import modules and load models

In [ ]:
# Imports

import gc
import math
import sys

import clip
from IPython import display
import k_diffusion as K
import torch
from torch import nn
from torchvision import utils
from torchvision.transforms import functional as TF
from tqdm.notebook import trange, tqdm

sys.path.append('/content/v-diffusion-pytorch')

from diffusion import get_model

In [ ]:
# Load the models

inner_model = get_model('cc12m_1_cfg')()
_, side_y, side_x = inner_model.shape
inner_model.load_state_dict(torch.load('v-diffusion-pytorch/checkpoints/cc12m_1_cfg.pth', map_location='cpu'))
inner_model = inner_model.half().cuda().eval().requires_grad_(False)
model = K.external.VDenoiser(inner_model)
clip_model = clip.load(inner_model.clip_model, jit=False, device='cpu')[0]

In [ ]:
#@title Settings

#@markdown The text prompt
prompt = 'New York City, oil on canvas'  #@param {type:"string"}

#@markdown The strength of the text conditioning (0 means don't condition on text, 1 means sample images that match the text about as well as the images match the text captions in the training set, 3+ is recommended).
weight = 5  #@param {type:"number"}

#@markdown Sample this many images.
n_images = 4  #@param {type:"integer"}

#@markdown Specify the number of diffusion timesteps (default is 50, can lower for faster but lower quality sampling).
steps =   50#@param {type:"integer"}

#@markdown The random seed. Change this to sample different images.
seed = 0  #@param {type:"integer"}

#@markdown Display progress every this many timesteps.
display_every =   10#@param {type:"integer"}

### Actually do the run...

In [ ]:
target_embed = clip_model.encode_text(clip.tokenize(prompt)).float().cuda()


class CFGDenoiser(nn.Module):
    def __init__(self, model, cond_scale):
        super().__init__()
        self.inner_model = model
        self.cond_scale = cond_scale

    def forward(self, x, sigma, clip_embed):
        x_in = torch.cat([x] * 2)
        sigma_in = torch.cat([sigma] * 2)
        clip_embed_in = torch.cat([torch.zeros_like(clip_embed), clip_embed])
        uncond, cond = self.inner_model(x_in, sigma_in, clip_embed=clip_embed_in).chunk(2)
        return uncond + (cond - uncond) * self.cond_scale


def callback(info):
    if info['i'] % display_every == 0:
        nrow = math.ceil(info['denoised'].shape[0] ** 0.5)
        grid = utils.make_grid(info['denoised'], nrow, padding=0)
        tqdm.write(f'Step {info["i"]} of {steps}, sigma {info["sigma"]:g}:')
        display.display(K.utils.to_pil_image(grid))
        tqdm.write(f'')


def run():
    gc.collect()
    torch.cuda.empty_cache()
    torch.manual_seed(seed)
    sigmas = K.sampling.get_sigmas_karras(steps, 1e-2, 160, device='cuda')
    x = torch.randn([n_images, 3, side_y, side_x], device='cuda') * sigmas[0]
    model_wrap = CFGDenoiser(torch.cuda.amp.autocast()(model), weight)
    extra_args = {'clip_embed': target_embed.repeat([n_images, 1])}
    outs = K.sampling.sample_lms(model_wrap, x, sigmas, extra_args=extra_args, callback=callback)
    tqdm.write('Done!')
    for i, out in enumerate(outs):
        filename = f'out_{i}.png'
        K.utils.to_pil_image(out).save(filename)
        display.display(display.Image(filename))


run()